# M31 2015 Luminous Red Nova, basic theoretical arguments

### Tassos Fragkos & Jeff J. Andrews

In this notebook, we are loading the static stellar profile of a 5.5 Msun subgiant. We will demonstrate, using this stellar profile, how energy is released over time as a companion spirals inside. Importantly, our results will indicate that the majority of the gravitational energy from the merger will be released near the core of the subgiant, and that it will take of order days for that energy to reach the stellar surface (based on the acoustic depth). This calculate justifies a more self-consistent approach involving the simultaneous modeling of the stellar structure as the companion star inspirals.

In [ ]:
# modules
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import odeint
import matplotlib.patches as mpatches

%matplotlib inline

In [ ]:
# constants in cgs
Gconst = 6.674e-8
Msun = 1.989e33
Rsun = 6.957e10
secday = 3600.0*24.0

### Load data and interpolate important variables

In [ ]:
data_dir = '../data/'

# # 5.5 Msun subgiant
# file_dir = data_dir + 'M31_2015_LNR_5.5M_40R_tau1.data.new'
# star = np.genfromtxt(file_dir, skip_header=5, names=True)

# 15 Msun giant
file_dir = data_dir + 'M31_2015_LNR_15M_250R_tau1.data'
star = np.genfromtxt(file_dir, skip_header=5, names=True)


star_m_encl = interp1d(star['radius']*Rsun, star['mass']*Msun)
star_log10_rho = interp1d(star['radius']*Rsun, star['logRho'])
star_pressure_scale_height = interp1d(star['radius']*Rsun, star['pressure_scale_height']*Rsun)
star_csound = interp1d(star['radius']*Rsun, star['csound'])
star_t_thermal_to_surf = interp1d(star['radius']*Rsun, 10**star['log_thermal_time_to_surface'])

# Create interpolation for dM_dr
dM_dr = star["dm"]/star["dr"]
star_dM_dr = interp1d(star['radius']*Rsun, dM_dr)
# dM_dr = np.zeros(len(star))
# i_max = len(star)-1
# for i in np.arange(i_max):
#     dM_dr[i] = (star["mass"][i] - star["mass"][i+1]) * Msun/((star["radius"][i] - star["radius"][i+1])*Rsun)
# dM_dr[i_max] = star["mass"][i_max] * Msun / (star["radius"][i_max] * Rsun)
# star_dM_dr = interp1d(star['radius'] * Rsun, dM_dr)


In [ ]:
# Test plots

r_test = np.linspace(min(star["radius"])*Rsun, max(star["radius"])*Rsun, 1000)

# Test plot for dM_dr
#dM_dr_test = star_dM_dr(r_test)
#plt.plot(r_test, dM_dr_test)
#plt.plot(r_test, star_dM_dr_1(r_test))
#plt.plot(r_test, (dM_dr_test - star_dM_dr_1(r_test))/star_dM_dr_1(r_test))
#plt.xlim(0, max(r_test))
#plt.ylim(0, 7e21)

# Test plot for pressure scale height
# Hp = star_pressure_scale_height(r_test)
# plt.plot(r_test, Hp)

# Test plot for density plot
# rho = 10**star_log10_rho(r_test)
# plt.plot(r_test,rho)
# plt.yscale('log')

# Test plot for mass
# mass = star_m_encl(r_test)
# plt.plot(r_test, mass)

#plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 8, 4
file_dir = data_dir + 'M31_2015_LRN_5.5M/LOGS/history.data'
star = np.genfromtxt(file_dir, skip_header=5, names=True)

xmin = 30
plt.plot(star['log_Teff'][xmin:], star['log_L'][xmin:], color='k')

R_mod = [2, 20, 30, 40, 60]
indices = []
labels = []
for R in R_mod:
    indices.append(np.argmin(np.abs(10**star['log_R']-R)))
    labels.append(str(int(10**star['log_R'][indices[-1]])) + ' R$_{\odot}$')
    # labels.append(str(int(10**star['log_R'][indices[-1]])))

plt.scatter(star['log_Teff'][indices], star['log_L'][indices], facecolors="None", edgecolor='k', s=80)
for i in np.arange(len(R_mod)):
    plt.text(star['log_Teff'][indices[i]]-0.02, star['log_L'][indices[i]]+0.02, labels[i])
    
plt.xlim(4.4, 3.4)
plt.ylim(2.6, 3.4)
plt.xlabel(r"Log T$_{\rm eff}$ (K)")
plt.ylabel("Log L (L$_{\odot}$)")

plt.savefig('../figures/static_HR.pdf')
#plt.show()

### Keplerian velocity function

In [ ]:
def get_keplerian_velocity(M_enclosed, M_companion, a_separation):
    """ Provides the keplerian velocity (circular orbit)
    
    Arguments
    ---------
    M_enclosed : float
        Enclosed mass of the companion, assumed to be a point (Msun)
    M_companion : float
        Companion mass (Msun)
    a_separation : float
        Orbital separation (Rsun)
        
    Returns
    -------
    v_orb : float
        circular orbital velocity (km/s)
    """
    
    v_squared = Gconst * (M_enclosed + M_companion) * Msun / (a_separation * Rsun)
    v_orb = np.sqrt(v_squared) / 1.0e5 # converted from cm/s to km/s
    
    return v_orb

### Gravitational potential

In [ ]:
def get_E_grav(M_enclosed, M_companion, a_separation):
    """ Provides the gravitational potential energy
    
    Arguments
    ---------
    M_enclosed : float
        Enclosed mass of the companion, assumed to be a point (Msun)
    M_companion : float
        Companion mass (Msun)
    a_separation : float
        Orbital separation (Rsun)
        
    Returns
    -------
    E_grav : float
        gravitational potential energy (erg)
    """

    E_grav = -Gconst * (M_enclosed*Msun) * (M_companion*Msun) / (a_separation*Rsun)
    
    return E_grav

### Orbital period

In [ ]:
def get_P_orb(M_enclosed, M_companion, a_separation):
    """ Provides the orbital period
    
    Arguments
    ---------
    M_enclosed : float
        Enclosed mass of the companion, assumed to be a point (Msun)
    M_companion : float
        Companion mass (Msun)
    a_separation : float
        Orbital separation (Rsun)
        
    Returns
    -------
    P_orb : float
        Orbital period (s)
    """

    vel = get_keplerian_velocity(M_enclosed, M_companion, a_separation) * 1.0e5 # velocity in cm/s
    circumference = 2.0 * np.pi * a_separation*Rsun
    
    P_orb = circumference / vel
    
    return P_orb

### Define the frictional drag force from MacLeod & Ramirez-Ruiz (2015)

In [ ]:
def get_F_drag(mass_in, vel_in, rho, scale_height):
    """ Provides the drag force from an inspiraling object
    
    Arguments
    ---------
    mass_in : float
        Companion mass (Msun)
    vel_in : float
        Companion velocity - assumed to be the Keplerian orbital (km/s)
    rho : float
        Local density (g/cm^3)
    scale_height : float
        Local pressure scale height (cm)
        
    Returns
    -------
    F_drag : float
        Drag force on the inspiraling companion (erg/s)
    """
    
    # Convert units to cgs
    mass = mass_in * Msun # Msun to g
    vel = vel_in * 1.0e5 # km/s to cm/s
        
    # Accretion radius
    R_accretion = 2.0 * Gconst * mass / vel**2
    
    # ratio of accretion radius to scale height
    eta = R_accretion / scale_height

    # Hoyle-Littleton drag
    F_drag_HL = np.pi * R_accretion**2 * rho * vel**2
    
    # Factor to adjust Hoyle-Littleton drag to MacLeod & Ramirez-Ruiz (2015) simulations
    f_factor = 1.91791946 + -1.52814698*eta + 0.75992092*eta**2
        
    # Drag force
    F_drag = F_drag_HL * f_factor
    
    return F_drag

### Plot enclosed mass, keplerian velocity as a function of radius

In [ ]:
# plt.rcParams['figure.figsize'] = 12, 8
# plt.rcParams['font.size'] = 10

# # set companion mass
# M_companion = 0.5

# def make_patch_spines_invisible(ax):
#     ax.set_frame_on(True)
#     ax.patch.set_visible(False)
#     for sp in ax.spines.values():
#         sp.set_visible(False)

# # Allow space on right side for multiple y-axes
# fig, host = plt.subplots()
# fig.subplots_adjust(right=0.75)

# # Keplerian velocity y-axis
# par1 = host.twinx()
# # Gravitational energy y-axis
# par2 = host.twinx()
# par2.spines["right"].set_position(("axes", 1.17))
# make_patch_spines_invisible(par2)
# par2.spines["right"].set_visible(True)
# # Drag force y-axis
# par3 = host.twinx()
# par3.spines["right"].set_position(("axes", 1.35))
# make_patch_spines_invisible(par3)
# par3.spines["right"].set_visible(True)
# # P_orb y-axis
# par4 = host.twinx()
# par4.spines["right"].set_position(("axes", 1.5))
# make_patch_spines_invisible(par4)
# par4.spines["right"].set_visible(True)


# # Set x-axis
# x = star['radius']
# host.set_xlabel('Radius (Rsun)')
# host.set_xlim(0.0, 40.0)

# # Create plotted functions
# M_encl, = host.plot(x, star['mass'], label="Enclosed mass", color='b')
# v_kep, = par1.plot(x, get_keplerian_velocity(star['mass'],M_companion,star['radius']), label="Keplerian velocity", color='r')
# E_grav, = par2.plot(x, -get_E_grav(star['mass'],M_companion,star['radius']), label="Gravitational energy", color='k')
# # Drag force
# velocity = get_keplerian_velocity(star['mass'],M_companion,star['radius'])
# rho = 10**star["logRho"]
# scale_height = star["pressure_scale_height"] * Rsun
# F_drag, = par3.plot(x, get_F_drag(M_companion, velocity, rho, scale_height), label="Drag force", color='g')
# # P_orb
# P_orb, = par4.plot(x, get_P_orb(star['mass'],M_companion,star['radius'])/secday, label="P_orb", color='c')

# # Set the y-axis labels
# host.set_ylabel('Enclosed mass (Msun)')
# par1.set_ylabel('Keplerian velocity (km/s)')
# par2.set_ylabel('-E_grav (erg)')
# par3.set_ylabel('Drag force (erg/s)')
# par4.set_ylabel('P_orb (days)')

# # Set the y-axis colors
# host.yaxis.label.set_color(M_encl.get_color())
# par1.yaxis.label.set_color(v_kep.get_color())
# par2.yaxis.label.set_color(E_grav.get_color())
# par3.yaxis.label.set_color(F_drag.get_color())
# par4.yaxis.label.set_color(P_orb.get_color())

# # Set the y-axis tick parameters
# tkw = dict(size=4, width=1.5)
# host.tick_params(axis='y', colors=M_encl.get_color(), **tkw)
# par1.tick_params(axis='y', colors=v_kep.get_color(), **tkw)
# par2.tick_params(axis='y', colors=E_grav.get_color(), **tkw)
# par3.tick_params(axis='y', colors=F_drag.get_color(), **tkw)
# par4.tick_params(axis='y', colors=P_orb.get_color(), **tkw)

# # Set the y-axis limits
# par1.set_ylim(0.0, 1000.0)
# par2.set_yscale('log')
# par2.set_ylim(1e49, 1e47)
# par3.set_yscale('log')
# par3.set_ylim(1e33, 1e38)

# plt.tight_layout()

# #plt.savefig('../figures/static_profile.pdf')
# plt.show()

### Let's try a multipanel version of the plot above

In [ ]:
# Load the star
data_dir = '../data/M31_2015_LRN_5.5M/LOGS/'

radii = np.array([])
stars = []

# 5.5 Msun star
n_profiles = len(os.listdir(data_dir))
print "Loading " + str(n_profiles) + " profiles..."
#print sorted(os.listdir(data_dir))

for profiles in os.listdir(data_dir):
    if 'data' in profiles and 'profile' in profiles:
        # print profiles
        file_dir = data_dir + profiles
        star = np.genfromtxt(file_dir, skip_header=5, names=True)
        stars.append(star) 
        radii = np.append(radii, star['radius'][0])

print "Finished loading profiles"

In [ ]:
fig, ax = plt.subplots(3, figsize=(8,8), sharex=True)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
secday = 3600.0*24.0

M_companion=0.5

# i = np.argmin(np.abs(radii-2.0))
# label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
# color='k'
# x = stars[i]["radius"]
# ax[0].plot(x, -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color, label=label)
# ax[1].plot(x, get_keplerian_velocity(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color)
# ax[1].plot(x, stars[i]['csound']/1.0e5, linestyle='--', color=color)
# ax[2].plot(x, stars[i]['acoustic_depth']/secday, color=color)


i = np.argmin(np.abs(radii-20.0))
label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
color='g'
x = stars[i]["radius"]
ax[0].plot(x, -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color, label=label)
ax[1].plot(x, get_keplerian_velocity(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color)
ax[1].plot(x, stars[i]['csound']/1.0e5, linestyle='--',color=color)
ax[2].plot(x, stars[i]['acoustic_depth']/secday, color=color)



i = np.argmin(np.abs(radii-30.0))
label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
color='r'
x = stars[i]["radius"]
ax[0].plot(x, -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color, label=label)
ax[1].plot(x, get_keplerian_velocity(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color)
ax[1].plot(x, stars[i]['csound']/1.0e5, linestyle='--',color=color)
ax[2].plot(x, stars[i]['acoustic_depth']/secday, color=color)



i = np.argmin(np.abs(radii-40.0))
label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
color='orange'
x = stars[i]["radius"]
ax[0].plot(x, -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color, label=label)
ax[1].plot(x, get_keplerian_velocity(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color)
ax[1].plot(x, stars[i]['csound']/1.0e5, linestyle='--',color=color)
ax[2].plot(x, stars[i]['acoustic_depth']/secday, color=color)



# i = np.argmin(np.abs(radii-50.0))
# label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
# color='r'
# x = stars[i]["radius"]
# ax[0].plot(x, -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color, label=label)
# ax[1].plot(x, get_keplerian_velocity(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color)
# ax[1].plot(x, stars[i]['csound']/1.0e5, linestyle='--',color=color)
# ax[2].plot(x, stars[i]['acoustic_depth']/secday, color=color)




i = np.argmin(np.abs(radii-60.0))
label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
color='b'
x = stars[i]["radius"]
ax[0].plot(x, -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color, label=label)
ax[1].plot(x, get_keplerian_velocity(stars[i]["mass"], M_companion, stars[i]["radius"]), color=color)
ax[1].plot(x, stars[i]['csound']/1.0e5, linestyle='--',color=color)
ax[2].plot(x, stars[i]['acoustic_depth']/secday, color=color)


# Legend
#ax[0].legend(ncol=2)
ax[0].legend()
v_kep, = plt.plot([], color='k', linestyle='-', label='Keplerian')
c_sound, = plt.plot([], color='k', linestyle='--', label='Sound speed')
ax[1].legend(handles=[v_kep, c_sound])

# Axis labels
ax[0].set_ylabel('Gravitational potential (erg/g)')
ax[1].set_ylabel('Velocity (km/s)')
ax[2].set_ylabel('Acoustic depth (days)')
ax[2].set_xlabel("Radius (R$_{\odot}$)")

# Y-axis
ax[0].set_yscale('log')
#ax[0].set_ylim(1.0e47,1.0e49)
ax[0].set_ylim(1.0e47, 5.0e48)
ax[1].set_ylim(0.0, 790.0)
ax[1].set_yticks([0, 200, 400, 600, ])
ax[2].set_ylim(0.0, 9.5)
ax[2].set_yticks([0, 2, 4, 6, 8])


plt.tight_layout()

fig.subplots_adjust(hspace=0)

#plt.savefig('../figures/static_profile.pdf')
plt.show()

### Now, let's define derivatives for the inspiral over the static stellar profile

In [ ]:
def get_dE_dr(E_in, r_in, M_companion, star_m_encl, star_dM_dr):
    """ Calculates the energy dissipation rate with respect to radius
    
    Arguments
    ---------
    E_in : float
        Total energy dissipated in star (erg)
    star_m_encl : interp1d object
        Enclosed mass interpolation object (Msun)
    star_dM_dr : interp1d object
        dM_dr interpolation object (g/cm)
    r_in : float
        The radius in question (cm)
    M_companion : float
        The companion mass to the star (Msun)
        
    Returns
    -------
    dE_dr : float
        The derivative of radius with respect to radius (erg/cm)    
    """
    
    # Calculate dE_dr
    m_encl = star_m_encl(r_in)
    dM_dr = star_dM_dr(r_in)
    dE_dr = -Gconst * dM_dr * (M_companion*Msun) / (2.0*r_in) + Gconst * (m_encl * (M_companion*Msun)) / (2.0*r_in**2)

    return dE_dr



    
def get_dE_dt(E_in, r_in, M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height):
    """ Calculates the energy dissipation rate
    
    Arguments
    ---------
    E_in : float
        Total energy dissipated in star (erg)
    r_in : float
        The radius in question (cm)
    M_companion : float
        The companion mass to the star (Msun)
    star_m_encl : interp1d object
        Enclosed mass interpolation object (Msun)
    star_log10_rho : interp1d object
        log10 density interpolation object (g/cm^3)
    star_pressure_scale_height : interp1d object
        Pressure scale height interpolation object (cm)
        
    Returns
    -------
    dE_dt : float
        The derivative of radius with respect to time (erg/s)    
    """
    
    # Calculate dE_dt
    m_encl = star_m_encl(r_in)
    velocity = get_keplerian_velocity(m_encl/Msun, M_companion, r_in/Rsun)
    rho = 10**star_log10_rho(r_in)
    scale_height = star_pressure_scale_height(r_in)
    dE_dt = -get_F_drag(M_companion, velocity, rho, scale_height) * (velocity*1.0e5)

    return dE_dt


    

def get_dr_dt(r_in, t_in, M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height, star_dM_dr):
    """ Calculates the inspiral time of the star
    
    Arguments
    ---------
    r_in : float
        The radius in question (cm)
    t_in : float
        Latent variable for derivative
    M_companion : float
        The companion mass to the star (Msun)
    star_m_encl : interp1d object
        Enclosed mass interpolation object (Msun)
    star_log10_rho : interp1d object
        log10 density interpolation object (g/cm^3)
    star_pressure_scale_height : interp1d object
        Pressure scale height interpolation object (cm)
    star_dM_dr : interp1d object
        dM_dr interpolation object (g/cm)
        
    Returns
    -------
    dr_dt : float
        The derivative of radius with respect to time (cm/s)
    """
    
    # Calculate dE_dt
    m_encl = star_m_encl(r_in)
    velocity = get_keplerian_velocity(m_encl/Msun, M_companion, r_in/Rsun)
    rho = 10**star_log10_rho(r_in)
    scale_height = star_pressure_scale_height(r_in)
    dE_dt = -get_F_drag(M_companion, velocity, rho, scale_height) * (velocity*1.0e5)
    
    # Calculate dE_dr
    dM_dr = star_dM_dr(r_in)
    dE_dr = -Gconst * dM_dr * (M_companion*Msun) / (2.0*r_in) + Gconst * (m_encl * (M_companion*Msun)) / (2.0*r_in**2)
    
    # dr_dt
    dr_dt = dE_dt / dE_dr
        
    return dr_dt
    

In [ ]:
plt.figure(figsize=(12,6))

r_test = star["radius"]*Rsun
time = 0.0

plt.plot(r_test/Rsun, get_dr_dt(r_test, time, M_companion, star_m_encl, star_log10_rho, \
                                star_pressure_scale_height, star_dM_dr) / 1.0e5, color='k')
plt.xlim(1, max(r_test/Rsun))
plt.ylim(-6e2, 0.0)

plt.xlabel(r'${\rm Radius}\ ({\rm R}_{\odot})$', size=20)
plt.ylabel(r'${\rm Radial\ velocity}\ \frac{dr}{dt}\ {\rm (km/s)}$',size=20)

plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(12,8), sharex=True)

tmin, tmax = 0.0, 4.25e5

# Radius evolution
r0 = 39.0*Rsun
t = []
t = np.linspace(tmin, tmax, 1000)
#t = np.linspace(0.0, 1.0e5, 100)
r_sol = odeint(get_dr_dt, r0, t, args=(M_companion,star_m_encl, star_log10_rho, \
                                       star_pressure_scale_height, star_dM_dr,))
ax[0].plot(t/secday, r_sol/Rsun, color='k')
ax[0].set_ylabel('Companion position (Rsun)')

# Energy
# use the results of the previous integration to get "r_sol" corresponding to x-axis "t"
E0 = 0.0
E_sol = odeint(get_dE_dr, E0, r_sol.T[0], args=(M_companion,star_m_encl, star_log10_rho,))
ax[1].plot(t/secday, -E_sol, color='k')
ax[1].set_yscale('log')
ax[1].set_xlim(tmin/secday, tmax/secday)
ax[1].set_ylim(8e44,3.0e48)
ax[1].set_ylabel('Cumulative E (erg)')


# Energy Dissipation
ax[2].plot(t/secday, -get_dE_dt(0.0, r_sol.T[0], M_companion, star_m_encl, star_log10_rho, \
                                star_pressure_scale_height), color='k')
ax[2].set_ylabel('Energy rate (erg/s)')
ax[2].set_yscale('log')
ax[2].set_ylim(8e40, 9e44)

ax[2].set_xlabel('Time (days)')

fig.subplots_adjust(hspace=0)

plt.savefig('../figures/static_inspiral.pdf')
#plt.show()

### Now, let's show how fast it takes for energy to reach the surface

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.rcParams['font.size'] = 14

t_thermal = 10**star['log_thermal_time_to_surface']
t_dynamical = 10**star['log_acoustic_depth']

#plt.plot(star['radius'], t_thermal/secday)
plt.plot(star['radius'], t_dynamical/secday, color='k')

plt.xlim(0.0, max(star['radius']))
plt.ylim(0.0, 5.0)

plt.ylabel('Acoustic depth (days)')
plt.xlabel('Radius (Rsun)')

plt.savefig('../figures/static_acoustic_depth.pdf')
#plt.show()

### Now, the gravitational potential of a 5.5 Msun star at different radii

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.rcParams['font.size'] = 14


#plt.title("Gravitational Potential")

i = np.argmin(np.abs(radii-15.0))
star = stars[i]
label = str(int(star['radius'][0])) + ' R$_{\odot}$'
plt.plot(star['radius']/star['radius'][0], -np.log10(-get_E_grav(star["mass"],M_companion,star["radius"])), color='k', label=label)

i = np.argmin(np.abs(radii-20.0))
star = stars[i]
label = str(int(star['radius'][0])) + ' R$_{\odot}$'
plt.plot(star['radius']/star['radius'][0], -np.log10(-get_E_grav(star["mass"],M_companion,star["radius"])), color='g', label=label)


i = np.argmin(np.abs(radii-30.0))
star = stars[i]
label = str(int(star['radius'][0])) + ' R$_{\odot}$'
plt.plot(star['radius']/star['radius'][0], -np.log10(-get_E_grav(star["mass"],M_companion,star["radius"])), color='y', label=label)


i = np.argmin(np.abs(radii-40.0))
star = stars[i]
label = str(int(star['radius'][0])) + ' R$_{\odot}$'
plt.plot(star['radius']/star['radius'][0], -np.log10(-get_E_grav(star["mass"],M_companion,star["radius"])), color='orange', label=label)


i = np.argmin(np.abs(radii-50.0))
star = stars[i]
label = str(int(star['radius'][0])) + ' R$_{\odot}$'
plt.plot(star['radius']/star['radius'][0], -np.log10(-get_E_grav(star["mass"],M_companion,star["radius"])), color='r', label=label)

i = np.argmin(np.abs(radii-60.0))
star = stars[i]
label = str(int(star['radius'][0])) + ' R$_{\odot}$'
plt.plot(star['radius']/star['radius'][0], -np.log10(-get_E_grav(star["mass"],M_companion,star["radius"])), color='b', label=label)


plt.ylim(-49, -47)
plt.yticks([-49, -48, -47])
#plt.ylim(1e47, 2e49)
#plt.yscale('log')


plt.xlabel('R/R$_{\star}$')
plt.ylabel('Log Gravitational Potential (erg)')

plt.legend(loc=4)

plt.savefig('../figures/static_potential.pdf')
#plt.show()

### Acoustic Depth

In [ ]:
def plot_t_dyn(star, color):
    t_dynamical = star['acoustic_depth']
    label = str(int(star['radius'][0])) + ' R$_{\odot}$'
    plt.plot(star['radius'], t_dynamical/secday, color=color, label=label)


fig = plt.figure(figsize=(8,4))
plt.rcParams['font.size'] = 14


# i = np.argmin(np.abs(radii-15.0))
# plot_t_dyn(stars[i], 'k')

i = np.argmin(np.abs(radii-20.0))
plot_t_dyn(stars[i], 'g')

i = np.argmin(np.abs(radii-30.0))
plot_t_dyn(stars[i], 'r')

i = np.argmin(np.abs(radii-40.0))
plot_t_dyn(stars[i], 'orange')

# i = np.argmin(np.abs(radii-50.0))
# plot_t_dyn(stars[i], 'r')

i = np.argmin(np.abs(radii-60.0))
plot_t_dyn(stars[i], 'b')



#plt.xlim(0.0, max(stars[0]['radius']))
plt.ylim(0.0, 10.0)

plt.ylabel('Acoustic depth (days)')
plt.xlabel('Radius (Rsun)')

plt.legend(loc=0)

plt.savefig('../figures/static_acoustic_depth.pdf')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(3, figsize=(12,12), sharex=True)
fig, ax = plt.subplots(2, figsize=(12,8), sharex=True)
plt.rcParams['font.size'] = 12


# Radius evolution
def get_companion_position(star, tmin, tmax, ax, color):
    r0 = 0.99*star['radius'][0] * Rsun
    t = []
    t = np.linspace(tmin, tmax, 1000)
    star_m_encl = interp1d(star['radius']*Rsun, star['mass']*Msun)
    star_log10_rho = interp1d(star['radius']*Rsun, star['logRho'])
    star_pressure_scale_height = interp1d(star['radius']*Rsun, star['pressure_scale_height']*Rsun)
    dM_dr = star["dm"]/star["dr"]
    star_dM_dr = interp1d(star['radius']*Rsun, dM_dr)
    r_sol = odeint(get_dr_dt, r0, t, args=(M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height, star_dM_dr))
    label = str(int(star['radius'][0])) + ' R$_{\odot}$'
    ax.plot(t/secday, r_sol/Rsun, color=color, label=label)

    
# Energy
# use the results of the previous integration to get "r_sol" corresponding to x-axis "t"
def get_energy_evolution(star, tmin, tmax, ax, color):
    r0 = 0.99*star['radius'][0] * Rsun
    E0 = 0.0
    t = []
    t = np.linspace(tmin, tmax, 1000)
    star_m_encl = interp1d(star['radius']*Rsun, star['mass']*Msun)
    star_log10_rho = interp1d(star['radius']*Rsun, star['logRho'])
    star_pressure_scale_height = interp1d(star['radius']*Rsun, star['pressure_scale_height']*Rsun)
    dM_dr = star["dm"]/star["dr"]
    star_dM_dr = interp1d(star['radius']*Rsun, dM_dr)
    r_sol = odeint(get_dr_dt, r0, t, args=(M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height, star_dM_dr))
    E_sol = odeint(get_dE_dr, E0, r_sol.T[0], args=(M_companion, star_m_encl, star_dM_dr))
    ax.plot(t/secday, -E_sol, color=color)


# Energy Dissipation
def get_energy_dissipation(star, tmin, tmax, ax, color):
    r0 = 0.99*star['radius'][0] * Rsun
    t = []
    t = np.linspace(tmin, tmax, 1000)
    star_m_encl = interp1d(star['radius']*Rsun, star['mass']*Msun)
    star_log10_rho = interp1d(star['radius']*Rsun, star['logRho'])
    star_pressure_scale_height = interp1d(star['radius']*Rsun, star['pressure_scale_height']*Rsun)
    dM_dr = star["dm"]/star["dr"]
    star_dM_dr = interp1d(star['radius']*Rsun, dM_dr)
    r_sol = odeint(get_dr_dt, r0, t, args=(M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height, star_dM_dr))
    ax.plot(t/secday, -get_dE_dt(0.0, r_sol.T[0], M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height), color=color)



# # R = 15 Rsun
# tmin, tmax = 0.0, 8.3e6
# i = np.argmin(np.abs(radii-15.0))
# get_companion_position(stars[i], tmin, tmax, ax[0], 'k')    
# get_energy_evolution(stars[i], tmin, tmax, ax[1], 'k')
# #get_energy_dissipation(stars[i], tmin, tmax, ax[2], 'k')


# R = 20 Rsun
tmin, tmax = 0.0, 1.011e7
i = np.argmin(np.abs(radii-20.0))
get_companion_position(stars[i], tmin, tmax, ax[0], 'g')    
get_energy_evolution(stars[i], tmin, tmax, ax[1], 'g')
#get_energy_dissipation(stars[i], tmin, tmax, ax[2], 'g')


# R = 30 Rsun
tmin, tmax = 0.0, 1.27e7
i = np.argmin(np.abs(radii-30.0))
get_companion_position(stars[i], tmin, tmax, ax[0], 'r')    
get_energy_evolution(stars[i], tmin, tmax, ax[1], 'r')
#get_energy_dissipation(stars[i], tmin, tmax, ax[2], 'r')

    
# R = 40 Rsun
tmin, tmax = 0.0, 4.25e5
i = np.argmin(np.abs(radii-40.0))
get_companion_position(stars[i], tmin, tmax, ax[0], 'orange')    
get_energy_evolution(stars[i], tmin, tmax, ax[1], 'orange')
#get_energy_dissipation(stars[i], tmin, tmax, ax[2], 'orange')


# # R = 50 Rsun
# tmin, tmax = 0.0, 4.0e5
# i = np.argmin(np.abs(radii-50.0))
# get_companion_position(stars[i], tmin, tmax, ax[0], 'r')    
# get_energy_evolution(stars[i], tmin, tmax, ax[1], 'r')
# #get_energy_dissipation(stars[i], tmin, tmax, ax[2], 'r')

    
# R = 60 Rsun
tmin, tmax = 0.0, 4.51e5
i = np.argmin(np.abs(radii-60.0))
get_companion_position(stars[i], tmin, tmax, ax[0], 'b')    
get_energy_evolution(stars[i], tmin, tmax, ax[1], 'b')
#get_energy_dissipation(stars[i], tmin, tmax, ax[2], 'b')






# Companion position plot
ax[0].set_ylim(1, 60)
ax[0].set_ylabel('Companion position (Rsun)')
ax[0].legend(loc=1)

# Cumulative energy plot
ax[1].set_ylim(1e44, 1e48)
ax[1].set_yscale('log')
ax[1].set_ylabel('Cumulative E (erg)')

# # Energy rate plot
# ax[2].set_ylim(1e39, 5e43)
# ax[2].set_ylabel('Energy rate (erg/s)')
# ax[2].set_yscale('log')
# ax[2].set_xlabel('Time (days)')




fig.subplots_adjust(hspace=0)

plt.savefig('../figures/static_inspiral.pdf')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(3, figsize=(12,12), sharex=True)
plt.rcParams['font.size'] = 12


radii_plot = 15, 20, 30, 40, 50, 60
colors_plot = 'k', 'g', 'y', 'orange', 'r', 'b'

    
for radius, color in zip(radii_plot, colors_plot):
    i = np.argmin(np.abs(radii-radius))
        
    r_set = np.linspace(0.01*stars[i]['radius'][0], 0.999*stars[i]['radius'][0], 1000)
    star_m_encl = interp1d(stars[i]['radius'], stars[i]['mass'])
    label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
    plt.plot(r_set, star_m_encl(r_set), color=color, label=label)
    

plt.xlabel('Radius (R$_{\star}$)')
plt.ylabel('Enclosed Mass (M$_{\odot}$)')

plt.legend(loc=4)
    
plt.savefig('../figures/static_mass_profile.pdf')
#plt.show()

### Now, let's compare binding and orbital energies as a function of radius

In [ ]:
i = np.argmin(np.abs(radii-30.0))


def plot_energies(i, color):
    
    label = str(int(stars[i]['radius'][0])) + ' R$_{\odot}$'
    
    # Orbital energy
    E_orb = -get_E_grav(stars[i]["mass"], M_companion, stars[i]["radius"])
    plt.plot(stars[i]['radius'], E_orb - E_orb[0], color=color, linestyle='solid', label=label)

    # Gravitational binding energy
    E_binding = np.zeros(len(stars[i]))
    for j in np.arange(len(stars[i])-1)+1:
        delta_E_binding = - get_E_grav(stars[i]["mass"][j], stars[i]['dm'][j]/Msun, stars[i]["radius"][j])
        E_binding[j] = E_binding[j-1] + delta_E_binding
    plt.plot(stars[i]['radius'], E_binding, color=color, linestyle='dashed')

    
    

radii_plot = 15, 20, 30, 40, 50, 60
colors_plot = 'k', 'g', 'y', 'orange', 'r', 'b'

for radius, color in zip(radii_plot, colors_plot):
    i = np.argmin(np.abs(radii-radius))
    plot_energies(i, color)

    
plt.legend()


plt.xlabel(r'Radius (R$_{\odot}$)')
plt.ylabel('-Energy (erg)')

plt.yscale('log')
plt.ylim(1.0e45, 2.0e49)


plt.tight_layout()
plt.savefig('../figures/Energy_orbital_binding.pdf')
#plt.show()

The above plot indicates that there is more orbital energy than binding energy in the outermost part of the envelope. This can be considered to work against the 1D approximation, since it indicates that there is enough energy in the orbit to dynamically disrupt the stellar envelope. 

Nevertheless, for stars with R>40 Rsun, it appears that very quickly the binding energy starts to dominate. This does argue for the 1D approximation in these stars, for the converse of the above argument, so long as the companion is able to plunge into the envelope faster than the outermost, weakly bound envelope layers do not escape from the system faster than the companion's orbit shrinks.

### Now, let's take a look at some timescales

In [ ]:
plt.figure(figsize=(12,6))

r_test = star["radius"]*Rsun
time = 0.0

M_companion = 1.5


def get_delta_t_therm(M_enclosed, M_companion, a_separation):

    vel = get_keplerian_velocity(M_enclosed, M_companion, a_separation) * 1.0e5 # velocity in cm/s
    R_accretion = 2.0 * Gconst * M_companion*Msun / vel**2

    t_thermal_low = star_t_thermal_to_surf(a_separation*Rsun)
    max_rad_array = star['radius'][0]*Rsun * np.ones(len(a_separation))
    t_thermal_high = star_t_thermal_to_surf(np.amin(np.vstack([a_separation*Rsun + R_accretion, max_rad_array]), axis=0))

    return t_thermal_high - t_thermal_low





dr_dt = get_dr_dt(r_test, time, M_companion, star_m_encl, star_log10_rho, star_pressure_scale_height, star_dM_dr)
P_orb = get_P_orb(star_m_encl(r_test)/Msun, M_companion, r_test/Rsun) / secday
plt.plot(r_test/Rsun, P_orb, label='Orbital Period')
plt.plot(r_test/Rsun, np.abs(r_test / dr_dt / secday), label=r'Inspiral timescale: $R$ / $\dot{R}$')



plt.plot(r_test/Rsun, np.pi/2. * r_test/star_csound(r_test) / secday, label=r'Polar timescale: $\frac{\pi}{2} \frac{r}{c_s}$')


# delta_t_thermal = get_delta_t_therm(star_m_encl(r_test)/Msun, M_companion, r_test/Rsun)
# plt.plot(r_test/Rsun, delta_t_thermal / secday, label=r'Local thermal timescale: $\Delta \tau_{\rm thermal}$')



# plt.plot(r_test/Rsun,  dr_dt / 1.0e5, color='k')
# plt.xlim(1, max(r_test/Rsun))
# plt.ylim(-6e2, 0.0)

plt.xlabel(r'${\rm Radius}\ ({\rm R}_{\odot})$', size=20)
plt.ylabel('Time (days)', size=20)
# plt.ylabel(r'${\rm Radial\ velocity}\ \frac{dr}{dt}\ {\rm (km/s)}$',size=20)

plt.xlim(0.1, 300.0)
plt.ylim(0.01, 100000)
plt.xscale('log')
plt.yscale('log')

plt.legend(loc=2)

plt.show()